__Автор__: Карпаев Алексей, ассистент кафедры информатики и вычислительной математики.

# Обзор средств ускорения Python

In [ ]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import numba as nb
import time

figsizeConst = (11.6, 7.)

## Некоторые способы
* cython
* scipy.weave
* SWIG
* ...
* __векторизация из Numpy__
* __Numba__

## Cython: 0.5 * C + 0.5 * Python
Пример кода (c), численное интегрирование:

In [ ]:
# cython не установлен
'''
cdef extern from "math.h":
    double sin(double x)

cdef double f(double x):
    return sin(x**2)

cpdef double integrate_f(double a, double b, int N):
    cdef double dx, s
    cdef int i

    dx = (b-a)/N
    s = 0
    for i in range(N):
        s += f(a+i*dx)
    return s * dx
'''

## Векторизация

Еще раз про "быстрые массивы" из __Numpy__:

* элементы массива должны быть одного типа
* неизменяемость длины массива
* поддержка векторизованных операций

### Принцип векторизованных операций: 
* задействование быстрых циклов на C, реализованных внутри библиотеки Numpy
* 1 вместо n инструкций для интерпретации в случае парсинга цикла

In [ ]:
def CalculateFibonacciNumber(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = b, a + b
    return a

$$ 
    t_{scalar} = (t_{interpret} + t_{execut1}) \cdot n \\
    t_{vectorized} = t_{interpret} + t_{execut2} n \\
    t_{execut1} \text{- время исполнения инструкции, переведенной в машинный код интерпретатором} \\
    t_{execut2} \text{- время исполнения инструкции, переведенной в машинный код компилятором языка С} \\
$$


* векторизованные операции Numpy - высокоуровневая "обертка" для реализованных внутри Numpy циклов на С

Массив $\mathbf{v} = \left( v_0, v_1, ..., v_{n-1} \right)$ 

* векторизованная функция: $\mathbf{f}(\mathbf{v}) = \left( f(v_0), f(v_1), ..., f(v_{n-1}) \right)$
* векторизованная арифметическая операция: $\mathbf{u} \circ \mathbf{v} = \left( u_0 \circ v_0 , u_1 \circ v_1, ..., u_{n-1} \circ v_{n-1} \right)$

Реализации покоординатных операций над массивами в явном виде:

In [ ]:
def AddArrays(array1, array2):
    n = len(array1)
    resultArray = [0. for i in range(n)]
    for i in range(n):
        resultArray[i] = array1[i] + array2[i]
    return resultArray

def MultiplyArrays(array1, array2):
    n = len(array1)
    resultArray = [0. for i in range(n)]
    for i in range(n):
        resultArray[i] = array1[i] * array2[i]
    return resultArray

def SpecificFunctionArray(array):
    n = len(array)
    resultArray = [0. for i in range(n)]
    for i in range(n):
        resultArray[i] = m.cos(m.sin(array[i]**2.)) / (1. + m.exp(-2.*array[i]))
    return resultArray

In [ ]:
N = int(1e3)
arrayA = [5*i for i in range(N)]
arrayB = list(arrayA)

print AddArrays(arrayA, arrayB)[1:10], '\n'
print MultiplyArrays(arrayA, arrayB)[1:10], '\n'
print SpecificFunctionArray(arrayB)[1:10], '\n'

In [ ]:
# график времени исполнения
lengthsList = [int(1.5**i) for i in range(2, 20)]
runtimesList = []

for length in lengthsList:
    arrayA = [5*i for i in range(int(length))]
    arrayB = list(arrayA)
    
    start = time.clock()
    numberOfRuns = 50
    for i in range(numberOfRuns):
        SpecificFunctionArray(arrayB)
    runtimesList.append(float(time.clock() - start)/numberOfRuns)
    
plt.figure()
plt.plot(lengthsList[:], runtimesList[:], 'b-o', linewidth=3, markersize = 7)
plt.ylabel('Runtime, s')
plt.xlabel('n')
plt.grid('on')
plt.show()

In [ ]:
# то же самое, с использованием векторизации

In [ ]:
N = int(1e3)
arrayA = np.array([5*i for i in range(N)])
arrayB = np.array(arrayA)

print arrayA[1:10], '\n'
print (arrayA[:] + arrayB[:])[1:10], '\n'
print (arrayA[:]*arrayB[:])[1:10], '\n'
print (np.cos(np.sin(arrayB[:]**2.)) / (1. + np.exp(-2.*arrayB[:])))[1:10], '\n'

In [ ]:
# график времени исполнения
lengthsList = [int(1.5**i) for i in range(2, 30)]
runtimesList = []

for length in lengthsList:
    arrayA, arrayB = np.array([5*i for i in range(int(length))]), np.array(arrayA)
    
    start = time.clock()
    numberOfRuns = 50
    for i in range(numberOfRuns):
        parrotsArray = np.cos(np.sin(arrayB[:]**2.)) / (1. + np.exp(-2.*arrayB[:]))
    runtimesList.append(float(time.clock() - start)/numberOfRuns)
    
plt.figure()
plt.plot(lengthsList[:], runtimesList[:], 'b-o', linewidth=3, markersize = 7)
plt.xlabel('n')
plt.ylabel('Runtime, s')
plt.grid('on')
plt.show()

In [ ]:
def SpecificFunctionArrayVectorized(array):
    return np.cos(np.sin(array[:]**2.)) / (1. + np.exp(-2.*array[:]))


def TimeImplementation(function, array, numberOfRuns):
    start = time.clock()
    for i in range(numberOfRuns):
        parrotsArray = function(array)
    return float(time.clock() - start)/numberOfRuns

In [ ]:
# графики ускорений
#lengthsList = [int(1.5**i) for i in range(2, 20)]
runtimesScalarList, runtimesVectorizedList = [], []


for length in lengthsList:
    arrayA, arrayB = np.array([5*i for i in range(int(length))]), np.array(arrayA)
    
    runtimesScalarList.append(TimeImplementation(SpecificFunctionArray, arrayB, 50))
    runtimesVectorizedList.append(\
                                                TimeImplementation(SpecificFunctionArrayVectorized, \
                                                                   arrayB,
                                                                   50))  

runtimesScalarList = np.array(runtimesScalarList) 
runtimesVectorizedList = np.asarray(runtimesVectorizedList)

plt.figure()
plt.title('Comparison')
plt.plot(lengthsList[:], runtimesScalarList[:]/runtimesVectorizedList[:], 'r-o', label='Vectorized', linewidth=3, markersize = 7)
plt.plot(lengthsList[:], runtimesScalarList[:]/runtimesScalarList[:], 'b-o', label='Scalar', linewidth=3, markersize = 7)
plt.ylabel('Speedup')
plt.xlabel('n')
plt.legend(loc='best')
plt.grid('on')
plt.show()

### Однородное линейное уравнение теплопроводности

In [ ]:
xLeft = 0.
xRight = 1.
kappa = 0.1
t = 0.
tRun = 0.04

def InitialCondition(x):
    if x < 0.6 and x > 0.4:
        return 1.
    else:
        return 0.


def SetInitialCondition(function, array, numPoints, h):
    for i in range(numPoints):
        array[i] = function(i * h)

def MakeStepScalar(uOld, numPoints, h, dt, kappa, uNew):
    courant = kappa * dt / (h * h)
    
    # явная реализация покоординатных операций с помощью цикла
    for i in range(1, numPoints - 1):
        uNew[i] = uOld[i] + courant * (uOld[i + 1] - 2 * uOld[i] + uOld[i - 1])
    
    
def MakeStepVectorized(uOld, numPoints, h, dt, kappa, uNew):
    courant = kappa * dt / (h * h) 
    
    # неявная реализация поокоординатных операций с помощью векторизации
    uNew[1:numPoints-1] = uOld[1:numPoints-1] + courant*(uOld[2:numPoints] - 2*uOld[1:numPoints-1]\
                                                        + uOld[:numPoints-2])

# main
counter = 0
counterBlocks = 0

numBlocks = int(1e3)
import time

start = time.time()
numPoints = int(numBlocks + 1)
uOld, uNew = np.zeros(numPoints), np.zeros(numPoints)
h = float(xRight - xLeft) / float(numBlocks)
dt = h**2 / (kappa * 20.)
SetInitialCondition(InitialCondition, uOld, numPoints, h)
    
while t < tRun:
    MakeStepScalar(uOld, numPoints, h, dt, kappa, uNew)
    #MakeStepVectorized(uOld, numPoints, h, dt, kappa, uNew)
    
    uOld = uNew
    t += dt; counter += 1
    if (counter % int(1e4)) == 0:
        print 'Step #%d' % counter

end = time.time()
runtime = (end - start)
print 'Calculations took ... %.2e s' % runtime
    
    
xArray = np.linspace(xLeft, xRight, numPoints)
plt.figure()
plt.title('Numerical solution')
plt.plot(xArray, uNew, 'b-o', linewidth=4)
plt.xlabel('X')
plt.ylabel('Temperature')
plt.grid('on')
plt.show()

### Численное интегрирование

In [ ]:
class AbstractIntegral:
    """ Class, performing numerical integration of a given function """
    
    def __init__(self):
        self._listOfCoefficients = []
        print 'An empty Integral is created.'  
    def ChooseFunction(self, function):
        self._function = function
    def SetLeftBorder(self, leftBorder):
        self._leftBorder = leftBorder   
    def SetGrid(self, step):
        self._step = step
    
    
    def SetMethodCoefficients(self):
        raise NotImplementedError
          
    def __call__(self, x):
        """ Function, performing the numerical integration via a loop """
        self._gridArray = np.arange(self._leftBorder, x, self._step)
        self._gridLength = len(self._gridArray)
        
        f, left, right, h = self._function, self._leftBorder, \
                            x, self._step
        
        #self._function = np.vectorize(self._function) # раскомментировать, если функция из модуля math
        self.SetMethodCoefficients()
        value = 0.
        start = time.clock()
        # вычисление интеграла
        print 'Computations took ...',
        value = np.dot(self._listOfCoefficients, self._function(self._gridArray))
        end = time.clock() - start
        print '%.2e sec' % end
        
        value *= self._step
        return value
    
    
    
class TrapeziumMethod(AbstractIntegral):
    def SetMethodCoefficients(self):
        self._listOfCoefficients = np.ones(self._gridLength)
        self._listOfCoefficients[0] *= 0.5
        self._listOfCoefficients[-1] *= 0.5

In [ ]:
# тестирование
listOfAntiderivatives = [TrapeziumMethod()]
listOfFunctions = [lambda t: np.exp(-t**2)] # для ускорения: функция должна быть из numpy, не из math

xLeft_, xRight_, step = 0., 3., 1e-4
for element, function in zip(listOfAntiderivatives, listOfFunctions):
    element.ChooseFunction(function)
    element.SetGrid(step)
    element.SetLeftBorder(xLeft_)
    print element(1.)

xData = np.arange(xLeft_ + 1e-3, xRight_, 5e-2)

for element in listOfAntiderivatives:
    yData = [element(x) for x in list(xData)]
    plt.figure()
    plt.plot(xData, yData, 'b-o', linewidth=4, markersize=5)
    plt.xlabel('X')
    plt.grid('on')
    plt.show()

### Метод Якоби решения СЛАУ
СЛАУ возникает в результате использования разностного шаблона "крест" при дискретизации уравнения Лапласа в квадратной области с граничными условиями Дирихле:

In [ ]:
numBlocksX = 100
numBlocksY = 100

length = 5.
height = 5.

xArray = np.linspace(0., length, numBlocksX)
yArray = np.linspace(0., height, numBlocksY)

hx = length / (numBlocksX - 1)
hy = height / (numBlocksY - 1)

p0 = np.zeros((numBlocksY, numBlocksX))

# условия на границах
p0[-1,:] = np.sin(1.5 * np.pi * xArray/xArray[-1])
p0[:,-1] = np.cos(1.5 * np.pi * yArray/yArray[-1])

In [ ]:
def SolveLaplace2D_JacobiScalar(p, numBlocksX, numBlocksY, l2Tolerance): 
    l2Norm = 1.
    pn = np.empty_like(p)
    iterations = 0
    
    start = time.clock()
    while l2Norm > l2Tolerance:
        pn = p.copy()
        
        # iterations
        for i in range(1, numBlocksX - 1):
            for j in range(1, numBlocksY - 1):
                p[i, j] = 0.25 * (pn[i, j + 1] + pn[i, j - 1] +\
                                      pn[i + 1, j] + pn[i - 1, j])
        
        numerator, denumenator = 0., 0.
        for i in range(numBlocksX):
            for j in range(numBlocksY): 
                numerator += (p[i, j] - pn[i, j])**2
                denumenator += pn[i, j]**2
            
        l2Norm = m.sqrt(numerator/denumenator)
        iterations += 1
    
    runtime = time.clock() - start
    print '%d iterations completed in scalar Jacobi solver; calculations took ... %.2f s' % (iterations, runtime)
    return p, runtime

In [ ]:
def SolveLaplace2D_JacobiVectorized(p, numBlocksX, numBlocksY, l2Tolerance):
    l2Norm = 1
    pn = np.empty_like(p)
    iterations = 0
    
    start = time.clock()
    while l2Norm > l2Tolerance:
        pn = p.copy()
        
        # iterations
        p[1:-1,1:-1] = 0.25 * (pn[1:-1,2:] + pn[1:-1,:-2] +\
                              pn[2:,1:-1] + pn[:-2,1:-1])
        
        l2Norm = np.sqrt(np.sum((p - pn)**2)/np.sum(pn**2))
        iterations += 1
    
    runtime = time.clock() - start
    print '%d iterations completed in vectorized Jacobi solver; calculations took ... %.2f s' % (iterations, runtime)
    return p, runtime

In [ ]:
tolerance = 1e-3
pScalar, runtimeScalar = SolveLaplace2D_JacobiScalar(p0.copy(), numBlocksX, numBlocksY, tolerance)
pVectorized, runtimeVectorized = SolveLaplace2D_JacobiVectorized(p0.copy(), numBlocksX, numBlocksY, tolerance)

print 'Speedup = %.0f' % (runtimeScalar/runtimeVectorized)

In [ ]:
# проверяем корректность численного решения
plt.figure(figsize = figsizeConst)
plt.title('Pressure')
im = plt.imshow(pVectorized, cmap = 'viridis', origin='lower')
cs = plt.contour(pVectorized)
plt.colorbar(im)
plt.show()

## Ускорение программ с помощью Numba

Простой пример:

In [ ]:
def CalculateFibonacciNumber(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = b, a + b
    return a


# добавляем спецификатор
@nb.jit()
def CalculateFibonacciNumberJITted(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = b, a + b
    return a

In [ ]:
n = int(1e6)
start = time.clock()
CalculateFibonacciNumber(n)
runtimeScalar = (time.clock() - start)
print 'Calculations took ... %.2e s' % runtimeScalar

In [ ]:
# реальное значение ускорения будет со 2-го запуска ячейки, когда функцию уже не требуется компилировать
start = time.clock()
CalculateFibonacciNumberJITted(n)
runtimeJITted = (time.clock() - start)
print 'Calculations took ... %.2e s' % runtimeJITted
print 'Speedup = %.0f' % (runtimeScalar/runtimeJITted)

Библиотека __Numba__ использует __JIT-компиляцию__ (Just-in-time compilation, компиляция «на лету»): динамическая компиляция (dynamic translation) — технология увеличения производительности программных систем, использующих байт-код, путём компиляции байт-кода в машинный код или в другой формат непосредственно во время работы программы. Таким образом достигается высокая скорость выполнения по сравнению с интерпретируемым байт-кодом [1] (сравнимая с компилируемыми языками) за счёт увеличения потребления памяти (для хранения результатов компиляции) и затрат времени на компиляцию.

__Ограничения__ для функций, к которым можно применять JIT-компиляцию:
* отсутсвие векторизованных операций Numpy
* использование Numpy разрешено только при инициализации массивов
* отсутсвие операций с высокоуровневыми структурами данных Python: со списками, кортежами, словарями, ...

__Numba__ может ускорять довольно сложные функции, но все же иногда ей это не удается. В этом случае она сгенерирует функцию, ничем не отличающуюся по произодительности от изначальной (без каких либо оповещений). Запуск Numba в nopython-режиме заставит библиотеку выдавать ошибку в случае неудачного исхода JIT-компиляции функции: "либо ускоренная функция, либо ничего":

In [ ]:
@nb.jit(nopython=True)
def CalculateFibonacciNumberJITted(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = b, a + b
    return a

### Работа с массивами
Добавляем спецификатор __@jit__ к функциям, в которых реализован скалярный код покоординатных операций над масиивами:

In [ ]:
@nb.jit(nopython=True)
def AddArraysJIT(array1, array2):
    n = len(array1)
    resultArray = np.zeros(n)
    for i in range(n):
        resultArray[i] = array1[i] + array2[i]
    return resultArray

@nb.jit(nopython=True)
def MultiplyArraysJIT(array1, array2):
    n = len(array1)
    resultArray = np.zeros(n)
    for i in range(n):
        resultArray[i] = array1[i] * array2[i]
    return resultArray

@nb.jit(nopython=True)
def SpecificFunctionArrayJIT(array):
    n = len(array)
    resultArray = np.zeros(n)
    for i in range(n):
        resultArray[i] = m.cos(m.sin(array[i]**2.)) / (1. + m.exp(-2.*array[i]))
    return resultArray

Ускорение JIT-скомпилированных функций по сравнению со скалярными и векторизованными:

In [ ]:
# реальное значение ускорения будет со 2-го запуска ячейки, когда функцию уже не требуется компилировать
# сравнение ускорений
runtimesScalarList, runtimesVectorizedList, runtimesJittedList = [], [], []


for length in lengthsList:
    arrayA, arrayB = np.array([5*i for i in range(int(length))]), np.array(arrayA)
    
    runtimesScalarList.append(TimeImplementation(SpecificFunctionArray, arrayB, 50))
    runtimesVectorizedList.append(\
                                                TimeImplementation(SpecificFunctionArrayVectorized, \
                                                                   arrayB,
                                                                   50))
    runtimesJittedList.append(TimeImplementation(SpecificFunctionArrayJIT, arrayB, 50))

runtimesScalarList = np.array(runtimesScalarList) 
runtimesVectorizedList = np.array(runtimesVectorizedList)
runtimesJittedList = np.array(runtimesJittedList)

plt.figure()
plt.title('Comparison')
plt.plot(lengthsList[1:], runtimesScalarList[1:]/runtimesVectorizedList[1:], 'r-o', label='Vectorized', linewidth=3, markersize = 7)
plt.plot(lengthsList[1:], runtimesScalarList[1:]/runtimesJittedList[1:], 'g-o', label='JITted', linewidth=3, markersize = 7)
plt.plot(lengthsList[1:], runtimesScalarList[1:]/runtimesScalarList[1:], 'b-o', label='Scalar', linewidth=3, markersize = 7)
plt.ylabel('Speedup')
plt.xlabel('N')
plt.legend(loc='best')
plt.grid('on')
plt.show()

### Однородное линейное уравнение теплопроводности

In [ ]:
# реальное значение ускорения будет со 2-го запуска ячейки, когда функцию уже не требуется компилировать

xLeft = 0.
xRight = 1.
kappa = 0.1
t = 0.
tRun = 0.04

def InitialCondition(x):
    if x < 0.6 and x > 0.4:
        return 1.
    else:
        return 0.


def SetInitialCondition(function, array, numPoints, h):
    for i in range(numPoints):
        array[i] = function(i * h)

def MakeStepScalar(uOld, numPoints, h, dt, kappa, uNew):
    courant = kappa * dt / (h * h)
    for i in range(1, numPoints - 1):
        uNew[i] = uOld[i] + courant * (uOld[i + 1] - 2 * uOld[i] + uOld[i - 1])
    
def MakeStepVectorized(uOld, numPoints, h, dt, kappa, uNew):
    courant = kappa * dt / (h * h) 
    uNew[1:numPoints-1] = uOld[1:numPoints-1] + courant*(uOld[2:numPoints] - 2*uOld[1:numPoints-1]\
                                                        + uOld[:numPoints-2])
    
@nb.jit(nopython=True) # раскомментировать
def MakeStepJIT(uOld, numPoints, h, dt, kappa, uNew):
    courant = kappa * dt / (h * h)
    for i in range(1, numPoints - 1):
        uNew[i] = uOld[i] + courant * (uOld[i + 1] - 2 * uOld[i] + uOld[i - 1])

# main
counter = 0
counterBlocks = 0

numBlocks = int(1e3)
import time

start = time.time()
numPoints = int(numBlocks + 1)
uOld, uNew = np.zeros(numPoints), np.zeros(numPoints)
h = float(xRight - xLeft) / float(numBlocks)
dt = h**2 / (kappa * 20.)
SetInitialCondition(InitialCondition, uOld, numPoints, h)
    
while t < tRun:
    MakeStepJIT(uOld, numPoints, h, dt, kappa, uNew)
    uOld = uNew
    t += dt; counter += 1
    if (counter % int(1e4)) == 0:
        print 'Step #%d' % counter

end = time.time()
runtime = (end - start)
print 'Calculations took ... %.2e s' % runtime
    
    
xArray = np.linspace(xLeft, xRight, numPoints)
plt.figure()
plt.title('Numerical solution')
plt.plot(xArray, uNew, 'b-o', linewidth=4)
plt.xlabel('X')
plt.ylabel('Temperature')
plt.grid('on')
plt.show()

### Метод Якоби

In [ ]:
# реальное значение ускорения будет со 2-го запуска ячейки, когда функцию уже не требуется компилировать

@nb.jit(nopython=True)
def SolveLaplace2D_JacobiJITted(p, pn, l2Tolerance):  # для получения вразумительного ускорения
                                                      # пришлось изменить список аргументов 
                                                      # и тело функции. 
                                                      # cм. подробности в руководстве к Numba 
    l2Norm = 1.
    numBlocksX, numBlocksY = p.shape
    iterations = 0
    
    #start = time.clock()
    while l2Norm > l2Tolerance:
        for i in range(1, numBlocksX - 1):
            for j in range(1, numBlocksY - 1):
                pn[i, j] = p[i, j]
        
        for i in range(1, numBlocksX - 1):
            for j in range(1, numBlocksY - 1):
                p[i, j] = 0.25 * (pn[i, j + 1] + pn[i, j - 1] +\
                                      pn[i + 1, j] + pn[i - 1, j])
        
        numerator, denumenator = 0., 0.
        for i in range(numBlocksX):
            for j in range(numBlocksY): 
                numerator += (p[i, j] - pn[i, j])**2
                denumenator += pn[i, j]**2
            
        l2Norm = np.sqrt(numerator/denumenator)
        iterations += 1
    
    #runtime = time.clock() - start
    #print '%d iterations completed; calculations took ... %.2f s' % (iterations, runtime)
    return p, iterations

In [ ]:
tolerance = 1e-3

start = time.clock()
pJITted, iterationsJITted = SolveLaplace2D_JacobiJITted(p0.copy(), p0.copy(), tolerance)
runtimeJITted = time.clock() - start
print '%d iterations completed in JITted Jacobi solver; calculations took ... %.2f s' % (iterationsJITted, runtimeJITted)

pVectorized, runtimeVectorized = SolveLaplace2D_JacobiVectorized(p0.copy(), numBlocksX, numBlocksY, tolerance)
pScalar, runtimeScalar = SolveLaplace2D_JacobiScalar(p0.copy(), numBlocksX, numBlocksY, tolerance)


print 'Speedup: JITted = %.0f; vectorized = %.0f;' % ((runtimeScalar/runtimeJITted),\
                                                      (runtimeScalar/runtimeVectorized))

In [ ]:
# проверяем корректность численного решения
plt.figure(figsize = figsizeConst)
plt.title('Pressure field')
im = plt.imshow(pJITted, cmap = 'viridis', origin='lower')
cs = plt.contour(pJITted)
plt.colorbar(im)
plt.show()

### Поддержка классов: "Work in progress" (с) Сontinuum Analytics

In [ ]:
real = nb.float32 # псевдоним для типа данных Numba

spec = [
    ('_x', real),      # a simple scalar field
    ('_y', real), 
    ('_length', real), # an array field
]


@nb.jitclass(spec)
class Vector(object):
    def __init__(self, x, y):
        self._x = x
        self._y = y

    def CalculateLength(self):
        self._length = np.sqrt(self._x**2 + self._y**2)
        return self._length

In [ ]:
vectos = Vector(5., 10.)
vectos.CalculateLength()

### Принципы ускорения программ на Python
* выделить самые ресурсоемкие участки кода (циклы по точкам расчетной сетки)
* обернуть эти участки в функции
* внутри функций ликвидировать операции со сложными структурами данных
* задействовать средства ускорения

$$
    \dots \dots \dots \dots
$$

Также: с помощью Numba можно программировать под __GPU [http://numba.pydata.org/numbadoc/dev/cuda/examples.html]:__

In [ ]:
# чтобы код стал рабочим требуется корректная установка библиотек для работы с Nvidia GPU
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

## Вопросы?

### При подготовке материалов курса использовалась следующая литература:
1. __H.P. Langtangen.__ A primer on scientific programming with Python.
2. __Шамин Р.В.__ Современные численные методы в объектно-ориентированном изложении на C#.

## Спасибо за внимание в течение семестра.